# (non-linear) model optimization

In [ ]:
import numpy as np
import scipy.stats as stats
import scipy.optimize as so
import matplotlib.pyplot as plt

## Data generating process

Let's consider the free fall of an object from rest, with a drag force proportional to the quadratic velocity $F_D = \beta v^2$. The drag force parameter is

$$
\beta = \frac{1}{2} C A \rho,
$$

where $C$ is the drag coefficient, $A$ the object's cross section area, and $\rho$ the density of the liquid therough which the object falls. With Newton's law of mation one can derive

$$
v(t) = v_T \tanh \left( \frac{v_T \beta}{m} t \right) \equiv v_T \tanh \left( \beta_t t \right), 
$$

where $\beta_t \equiv \frac{v_T \beta}{m}$ and $v_T$ is the terminal velocity

$$
v_T = \sqrt{ \frac{2 m g}{\rho C A} }.
$$

Consider a 75-kg skydiver descending head first. We estimate the area $A = 0.18 \mathrm{m}^2$ and a drag coefficient of approximately $C \approx 0.7$. Assume the density of air is $\rho = 1.21 \mathrm{kg m}^{-3}$. With these parameter values we find that the terminal velocity is

$$
v_T = \sqrt{ \frac{2 (75 \mathrm{kg}) (9.80 \mathrm{m/s}^2)}{(1.21 \mathrm{kg m}^{-3}) (0.70) (0.18 \mathrm{m}^2)} } = 98 \mathrm{m/s},
$$

and

$$
\beta_t = \frac{(98 \mathrm{m/s}) 0.5 (0.70) (0.18 \mathrm{m}^2) (1.21 \mathrm{kg m}^{-3})}{(75 \mathrm{kg})} = 0.1 \mathrm{s}^{-1}
$$

In [ ]:
class DataGeneration():
    def __init__(self, Nd=8, tmin=0.0, tmax=10.0, error=0.1, seed=None):
        self.rng=np.random.default_rng(seed=seed)
        self.tdata = np.sort(self.rng.uniform(low=tmin, high=tmax, size=(Nd,1)),axis=0)
        self.mass = 75
        self.g = 9.80
        self.rho = 1.21
        self.A = 0.18
        # Introduce a randomness when setting the drag coefficient
        self.C = self.rng.uniform(low=0.5, high=0.9)
        self.v_T = np.sqrt( 2 * self.mass * self.g / (self.rho * self.C * self.A) )
        self.beta = 0.5 * self.C * self.A * self.rho
        self.true_params = self.v_T * np.array([ 1, self.beta / self.mass])
        # Measurement error
        self.sigma_error=error
        self.vdata = self.measurement()
        
    def true_model(self, params, tdata):
        vtrue = params[0] * np.tanh(params[1]*tdata)
        return vtrue

    def measurement(self):
        vdata = self.true_model(self.true_params, self.tdata)
        verror = self.rng.normal(0,self.sigma_error,len(self.tdata)).reshape(-1,1)
        return vdata + verror

In [ ]:
tmin=0.0
tmax=10.
Nd=8
seed=42
process = DataGeneration(Nd=Nd, tmin=tmin, tmax=tmax, seed=seed)

In [ ]:
process.true_params

In [ ]:
process.tdata

In [ ]:
fig,ax = plt.subplots(1,1)
ax.errorbar(process.tdata.flatten(),process.vdata.flatten(),yerr=process.sigma_error,fmt='o',label=r'$\mathcal{D}$')
t_plot = np.linspace(tmin,tmax,100)
v_plot = process.true_model(process.true_params, t_plot)
ax.plot(t_plot, v_plot, 'r-', label='True model')
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$v$')
ax.legend(loc='best');

## Optimize model
Assume first that we know the true model which enters the data generating process. We want to optimize the model parameters to best fit the observed data.

We will minimize the least-squares cost function using a non-linear optimization method from `scipy.optimize`. Specifically we will use the [Levenberg-Marquardt algorithm](https://en.wikipedia.org/wiki/Levenberg–Marquardt_algorithm) in which the gradient is not provided but will be estimated in the iterative optimization.

In [ ]:
def model_true(theta, t):
    "evaluate our model at t given parameters theta"
    return theta[0] * np.tanh(theta[1] * t)

def residuals(theta, tdata, vdata, model):
    "return a vector of residuals for given data, model, and model parameters (1st argument)"
    v_model = model(theta, tdata)
    return v_model - vdata

In [ ]:
theta_start = np.array([100,0.1])
# Read the manual of `scipy.optimize.least_squares`
# Here we use the Levenberg-Marquardt method
res_lsq = so.least_squares(residuals, theta_start, method='lm', \
                           args=(process.tdata.flatten(), process.vdata.flatten(), model_true))
print(res_lsq)

In [ ]:
print('       best fit: ', res_lsq.x)
print('true parameters: ', process.true_params)
print('  cost function: ', res_lsq.cost)

## Optimize complex model
Assume instead that we use a too complex model with an extra parameter $\beta_2$.

$$
v(t) = v_T \tanh \left( \beta_t t \right) \left( 1 - e^{-\beta_2 t} \right)
$$

In [ ]:
def model_complex(theta, t):
    "evaluate our model at t given parameters theta. len(theta)=3"
    return theta[0] * np.tanh(theta[1] * t) * (1 - np.exp(-theta[2]*t))

In [ ]:
theta_start = np.array([100,0.07, 10])
res_lsq_complex = so.least_squares(residuals, theta_start, method='lm',\
                                   args=(process.tdata.flatten(), process.vdata.flatten(), model_complex))
print(res_lsq_complex)

In [ ]:
print('       best fit: ', res_lsq_complex.x)
print('true parameters: ', process.true_params)
print('  cost function: ', res_lsq_complex.cost)

Explore and reflect on:
- the choice of starting guess
- the value of the cost function at the optimum
- the values of the optimum parameters
- assuming that the goal is to infer the terminal velocity, what do you conclude?